In [152]:
import torch
from cgan_code import Generator  # 假设你有一个定义好的 Discriminator 类

In [153]:
NPRINT_LINE_LEN = 114
# NPRINT_REAL_WIDTH = 50*8
NPRINT_REAL_WIDTH = 22*8
# LABEL_DICT = {'facebook': 0, 'aim': 1, 'email': 2, 'voipbuster': 3, 'hangouts': 4, 'youtube': 5, 'sftp': 6, 'icq': 7, 'skype': 8, 'ftps': 9, 'vimeo': 10, 'spotify': 11, 'netflix': 12, 'bittorrent.json': 13}
LABEL_DICT = {'facebook': 0, 'skype': 1}
BYTES_LOG_MEAN = 5.961416185547372
BYTES_LOG_STD = 1.4073518072041993
PACKETS_LOG_MEAN = 1.5394048328338192
PACKETS_LOG_STD = 1.1067132728539024

SEQ_DIM = 4
MAX_PKT_LEN = 1500
MAX_TIME = 1000
MAX_PORT = 65536
MAX_SEQ_LEN = 16

label_dim = len(LABEL_DICT) 
image_dim = (1, NPRINT_REAL_WIDTH, NPRINT_REAL_WIDTH)  # 生成单通道图像
# metadata_dim = 2+16+2  # 元数据维度为2
noise_dim = 100  # 噪声维度
batch_size = 64
epochs = 200

In [154]:
# 创建模型实例（与保存时的模型架构相同）
generator = Generator(label_dim, noise_dim, SEQ_DIM, MAX_SEQ_LEN, 'cpu')

# 加载模型权重
checkpoint = torch.load('./save_new/generator_350.pth')  # 加载保存的权重字典
generator.load_state_dict(checkpoint)  # 将权重字典加载到模型中

# 切换到评估模式
generator.eval()

/tmp/ipykernel_190271/3796080591.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('./save_new/generator_350.pth')  # 加载保存的权重字典


Generator(
  (label_fc): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU(inplace=True)
  )
  (noise_fc): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU(inplace=True)
  )
  (combine_fc): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): ReLU(inplace=True)
  )
  (lstm): LSTM(512, 512, num_layers=4, batch_first=True)
  (output_layer): Sequential(
    (0): Linear(in_features=512, out_features=4, bias=True)
    (1): Tanh()
  )
)

In [155]:
labels = torch.zeros(batch_size, label_dim).to('cpu')  # 初始化为全 0
labels[:, 1] = 1  # 将每个样本的第一个值设置为 1，形成 one-hot 编码    
noise = torch.randn(batch_size, noise_dim).to('cpu') 
lengths = torch.zeros(batch_size).to('cpu')
lengths.fill_(16)
seqs = torch.clamp(generator(labels, noise, lengths),-1,1)

In [156]:
id = 0
for seq in seqs.tolist():
    for i in range(min(int(lengths.tolist()[id]),MAX_SEQ_LEN)):
        [time,pkt_len,dport,sign] = seq[i]
        # print(time,pkt_len,dport,sign)
        
        
        time = (time + 1)/2*MAX_TIME
        pkt_len = (pkt_len + 1)/2*MAX_PKT_LEN
        dport = (dport + 1)/2*MAX_PORT
        # print(time,pkt_len,dport)

        time = round(time,2)
        pkt_len = round(pkt_len)
        dport = round(dport)
        if sign > 0:
            pkt_len = - pkt_len
        print(time,pkt_len,dport)
    id+=1
    print()

0.22 100 22044
2.0 -139 39115
70.04 213 19842
13.61 111 17655
0.01 -61 17724
45.93 243 18973
43.93 116 20672
0.0 -52 10324
44.9 245 19018
46.84 116 20922
0.0 -54 9638
45.72 244 19034
44.96 116 20766
0.0 -53 10088
45.18 244 19023
46.19 116 20868

0.57 76 36984
1.44 -135 38812
67.39 216 19675
15.14 111 17792
0.01 -58 17009
45.27 244 18972
45.51 116 20807
0.0 -53 9949
45.32 244 19026
45.88 116 20842
0.0 -53 9870
45.43 244 19028
45.61 116 20820
0.0 -53 9933
45.36 244 19026
45.78 116 20835

0.38 81 30003
1.38 -137 38690
67.45 216 19672
15.06 111 17782
0.01 -58 17033
45.3 244 18972
45.45 116 20802
0.0 -53 9965
45.3 244 19025
45.92 116 20846
0.0 -53 9860
45.45 244 19028
45.59 116 20818
0.0 -53 9940
45.35 244 19026
45.8 116 20836

0.53 81 36273
1.47 -136 38854
67.52 216 19683
15.06 111 17784
0.01 -58 17043
45.3 244 18972
45.44 116 20801
0.0 -53 9968
45.3 244 19025
45.93 116 20847
0.0 -53 9858
45.45 244 19028
45.58 116 20818
0.0 -53 9941
45.35 244 19026
45.81 116 20836

0.74 74 40595
1.47 -135 